In [40]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import os, random
import scipy.ndimage
import matplotlib.pyplot as plt
import dicom_numpy

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '../Datasets/DICOM Native anonymized'
over_patients = os.listdir(INPUT_FOLDER)
# patients.sort()
print(over_patients, len(over_patients))

['1.2.124.113532.12.10699.12127.20150930.134304.394576165', '1.2.124.113532.12.10699.12127.20140805.115557.375988118', '1.3.12.2.1107.5.1.4.53642.30000011020807151056200000013', '1.3.12.2.1107.5.1.4.36902.30000008110707525750000000013', '1.2.124.113532.12.10699.12127.20150327.92216.826029339', '1.2.840.113619.2.81.290.46036.3820.20141013.202435', '1.2.124.113532.12.10699.12127.20151016.73704.433945372', '1.3.12.2.1107.5.1.4.53642.30000010121407173196800000064', '1.2.124.113532.12.10699.12127.20131129.133812.441560837', '1.2.124.113532.12.10699.12127.20130107.131930.52682931', '1.2.124.113532.12.10699.12127.20130716.75845.221118257', '1.3.12.2.1107.5.1.4.53013.4.0.3993252427261330', '1.2.124.113532.12.10699.12127.20120326.124951.95910468', '1.3.12.2.1107.5.1.4.53013.30000007091709141164000001004', '1.2.124.113532.12.10699.12127.20150409.131558.850932664', '1.2.124.113532.12.10699.12127.20110513.80519.17850612', '1.2.124.113532.12.10699.12127.20140721.114446.350220518', '1.2.392.200036.9

In [10]:
def get_patient_target_folder(patient_parent_folder):
    patient_target_folder = os.listdir(patient_parent_folder)[0]
    patient_target_folder = patient_parent_folder + '/' + patient_target_folder
    return patient_target_folder

In [11]:
def get_list_patients_paths(input_folder):
    list_patients_paths = []
    for element in os.listdir(input_folder):
        patient_parent_folder = os.path.join(INPUT_FOLDER, element)
        patient_target_folder = get_patient_target_folder(patient_parent_folder)
        list_patients_paths.append(patient_target_folder)
        # print(patient_target_folder)
    return list_patients_paths

list_patients_paths = get_list_patients_paths(INPUT_FOLDER)
print(list_patients_paths)


['../Datasets/DICOM Native anonymized/1.2.124.113532.12.10699.12127.20150930.134304.394576165/1.2.392.200036.9116.2.6.1.3268.2047269746.1444918787.14638', '../Datasets/DICOM Native anonymized/1.2.124.113532.12.10699.12127.20140805.115557.375988118/1.3.12.2.1107.5.1.4.53642.30000014101706044771800011216', '../Datasets/DICOM Native anonymized/1.3.12.2.1107.5.1.4.53642.30000011020807151056200000013/1.3.12.2.1107.5.1.4.53642.30000011020806384268700000583', '../Datasets/DICOM Native anonymized/1.3.12.2.1107.5.1.4.36902.30000008110707525750000000013/1.3.12.2.1107.5.1.4.36902.30000008110707190282800000330', '../Datasets/DICOM Native anonymized/1.2.124.113532.12.10699.12127.20150327.92216.826029339/1.2.392.200036.9116.2.6.1.3268.2047269746.1430895625.573756', '../Datasets/DICOM Native anonymized/1.2.840.113619.2.81.290.46036.3820.20141013.202435/1.2.840.113619.2.81.290.46036.3820.2.20141013.202524', '../Datasets/DICOM Native anonymized/1.2.124.113532.12.10699.12127.20151016.73704.433945372/1.2

In [ ]:
# source: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
# source: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
# sample_patient = list_patients_paths[1]
# sample_patient = '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110321.143718.22145985/1.3.12.2.1107.5.1.4.53642.30000011032114095153100000518'
# sample_patient = 'C:/Users/User/Documents/DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110321.143718.22145985/1.3.12.2.1107.5.1.4.53642.30000011032114095153100000518'
# first_patient = load_scan(sample_patient)
# first_patient_pixels = get_pixels_hu(first_patient)
# plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
# plt.xlabel("Hounsfield Units (HU)")
# plt.ylabel("Frequency")
# plt.show()

# # Show some slice in the middle
# plt.imshow(first_patient_pixels[0], cmap=plt.cm.gray)
# plt.show()

In [48]:
def show_dcm_info(dataset):
#     print("Filename.........:", file_path)
    print("Storage type.....:", dataset.SOPClassUID)
    print()

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    print("Patient id..........:", dataset.PatientID)
    print("Patient's Age.......:", dataset.PatientAge)
    print("Patient's Sex.......:", dataset.PatientSex)
    print("Modality............:", dataset.Modality)
    print("Body Part Examined..:", dataset.BodyPartExamined)
#     print("View Position.......:", dataset.ViewPosition)
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

def plot_pixel_array(dataset, figsize=(10,10)):
    plt.figure(figsize=figsize)
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()

In [49]:
# for patient_path in list_patients_paths:
#     slices = [pydicom.read_file(patient_path + '/' + s) for s in os.listdir(patient_path)]
#     slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
#     try:
#         print(slices[0])
#     except:
#         print("Pixel array not found")
#     #plt.imshow(slices[80].pixel_array)
#     #plt.show()

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.392.200036.9116.2.6.1.3268.2047269746.1444918810.116786
(0008, 0020) Study Date                          DA: '20151015'
(0008, 0021) Series Date                         DA: '20151015'
(0008, 0022) Acquisition Date                    DA: '20151015'
(0008, 0023) Content Date                        DA: '20151015'
(0008, 0030) Study Time                          TM: '161747.000000'
(0008, 0031) Series Time                         TM: '161914.681'
(0008, 0032) Acquisition Time                    TM: '161932.150'
(0008, 0033) Content Time                        TM: '161940.180'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0070) Manufacturer   

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.36902.30000008110707190282800000353
(0008, 0020) Study Date                          DA: '20081107'
(0008, 0021) Series Date                         DA: '20081107'
(0008, 0022) Acquisition Date                    DA: '20081107'
(0008, 0023) Content Date                        DA: '20081107'
(0008, 0030) Study Time                          TM: '095852.343000'
(0008, 0031) Series Time                         TM: '100030.000000'
(0008, 0032) Acquisition Time                    TM: '100054.893030'
(0008, 0033) Content Time                        TM: '100054.893030'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113619.2.81.290.46036.3820.2.40.20141013.202703
(0008, 0020) Study Date                          DA: '20141013'
(0008, 0021) Series Date                         DA: '20141013'
(0008, 0022) Acquisition Date                    DA: '20141013'
(0008, 0023) Content Date                        DA: '20141013'
(0008, 0030) Study Time                          TM: '102435.000000'
(0008, 0031) Series Time                         TM: '102524'
(0008, 0032) Acquisition Time                    TM: '102542.737196'
(0008, 0033) Content Time                        TM: '102703'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0070) Manufacturer            

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.53642.30000013112906510631200006908
(0008, 0020) Study Date                          DA: '20131129'
(0008, 0021) Series Date                         DA: '20131129'
(0008, 0022) Acquisition Date                    DA: '20131129'
(0008, 0023) Content Date                        DA: '20131129'
(0008, 0030) Study Time                          TM: '135425.156000'
(0008, 0031) Series Time                         TM: '135903.328000'
(0008, 0032) Acquisition Time                    TM: '135922.210599'
(0008, 0033) Content Time                        TM: '135922.210599'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.53642.30000013071605200709300002760
(0008, 0020) Study Date                          DA: '20130716'
(0008, 0021) Series Date                         DA: '20130716'
(0008, 0022) Acquisition Date                    DA: '20130716'
(0008, 0023) Content Date                        DA: '20130716'
(0008, 0030) Study Time                          TM: '104248.406000'
(0008, 0031) Series Time                         TM: '104757.750000'
(0008, 0032) Acquisition Time                    TM: '104825.377710'
(0008, 0033) Content Time                        TM: '104825.377710'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0

InvalidDicomError: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.

In [50]:
random_patient = random.choice(list_patients_paths)
random_slice = random.choice(os.listdir(random_patient))
random_slice_path = os.path.join(random_patient, random_slice)

# sample_dicom_file = '../Datasets/DICOM Native anonymized/1.2.124.113532.12.10699.12127.20110301.154346.9227917/1.3.12.2.1107.5.1.4.53642.30000011022806271114000017842/1.3.12.2.1107.5.1.4.53642.30000011022806271114000017931.dcm'
sample_slice = pydicom.read_file(random_slice_path)
show_dcm_info(sample_slice)

Storage type.....: 1.2.840.10008.5.1.4.1.1.2

Patient's name......: Volume_Measurements_JP, 
Patient id..........: 1778010367
Patient's Age.......: 048Y
Patient's Sex.......: M
Modality............: CT
Body Part Examined..: 
Image size.......: 512 x 512, 185934 bytes
Pixel spacing....: [0.702, 0.702]


In [33]:
print(sample_slice.pixel_array)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fd38346d3b0>

In [ ]:
for patient_path in list_patients_paths:
    try:
        slices = [pydicom.read_file(patient_path + '/' + s) for s in os.listdir(patient_path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
#         print(slices[0])
        print()
    except:
        print("Pixel array not found")

In [28]:
random_slice_path

'../Datasets/DICOM Native anonymized/1.3.12.2.1107.5.1.4.36902.30000008110707525750000000013/1.3.12.2.1107.5.1.4.36902.30000008110707190282800000330/1.3.12.2.1107.5.1.4.36902.30000008110707190282800000349.dcm'